# Continuation-Passing Style (CPS): A Translation View

This note gives a **minimal, rule-based view** of continuation-passing style
(CPS). The goal is that after reading it, you should be able to **mechanically
translate** small direct-style programs into CPS, and see how the same pattern
extends to exceptions and backtracking.

We focus on:

- direct style vs CPS
- a *general translation scheme* for expressions and statements
- exceptions via CPS (success + error continuations)
- a small backtracking example

## 1. Direct style vs CPS: first contact

In *direct style* we write functions that compute a result and return it:

```python
def inc(x):
    return x + 1

def twice(x):
    return inc(inc(x))
```

In *continuation-passing style* (CPS), a function **never returns**. Instead,
it receives an extra argument `k` (the *continuation*) that represents
"what to do with the result". It calls `k(result)` instead of `return result`.

In [1]:
def inc_direct(x):
    return x + 1

def inc_cps(x, k):
    # k is the continuation: what to do with x+1
    k(x + 1)


def twice_direct(x):
    return inc_direct(inc_direct(x))

def twice_cps(x, k):
    # first increment x, then increment again, then pass to k
    inc_cps(x, lambda v1: inc_cps(v1, k))


def show(v):
    print("result:", v)

print("direct:", twice_direct(10))
twice_cps(10, show)

direct: 12
result: 12


Key points:

- A CPS function takes an extra argument `k`.
- It **does not** `return`; it **calls `k(value)`** instead.
- To *run* a CPS function, you pass a **final continuation** (like `show`).

## 2. A general scheme for translating to CPS

We sketch simple **translation rules** that you can apply mechanically.

We restrict to expressions and functions; this already covers a large
class of programs.

### 2.1. Rule: constants and variables

Direct style:

```python
42
x
```

CPS form, given a continuation `k`:

```python
k(42)
k(x)
```

We always *call* the continuation with the value.

### 2.2. Rule: binary operations

Direct style:

```python
x + y
```

CPS form, assuming `x` and `y` are already values:

```python
k(x + y)
```

If `x` and `y` themselves come from computations, we CPS-transform those
computations and chain them using nested continuations.

### 2.3. Rule: function definition and call

Direct style:

```python
def f(x):
    return e

result = f(10)
```

CPS form:

1. Translate the body of `f`:

   - direct: `return e`
   - CPS:    `CPS(e, k)` i.e. "evaluate e then call k on the result"

2. The CPS version of `f` takes an extra continuation argument:

```python
def f_cps(x, k):
    # CPS(e, k)
    ...
```

3. A call `f(10)` in direct style becomes `f_cps(10, k)` in CPS.

In [2]:
def f_direct(x):
    return (x + 1) * 2

def f_cps(x, k):
    # mechanically: compute (x+1)*2 then call k
    k((x + 1) * 2)


def example_direct():
    return f_direct(10)

def example_cps(k):
    # translate "return f_direct(10)" as:
    f_cps(10, k)

print("example_direct:", example_direct())
example_cps(lambda v: print("example_cps:", v))

example_direct: 22
example_cps: 22


Note: We did *not* change the arithmetic; we only replaced `return` with
"call the continuation". In more complex programs, we systematically introduce
continuations at each call site.

### 2.4. Rule: sequencing with intermediate variables

Consider a simple sequence of computations:

```python
def h(x):
    y = f(x)
    z = g(y)
    return z + 1
```

A CPS translation has to:

1. Call `f` in CPS form.
2. In its continuation, call `g`.
3. In its continuation, add 1 and call the final continuation.

In [3]:
def f_direct(x):
    return x + 1

def g_direct(y):
    return y * 2

def h_direct(x):
    y = f_direct(x)
    z = g_direct(y)
    return z + 1

def f_cps(x, k):
    k(x + 1)

def g_cps(y, k):
    k(y * 2)

def h_cps(x, k):
    # CPS translation of "y = f(x); z = g(y); return z + 1"
    f_cps(x, lambda y:
          g_cps(y, lambda z:
                k(z + 1)))

print("h_direct:", h_direct(10))
h_cps(10, lambda v: print("h_cps:", v))

h_direct: 23
h_cps: 23


The CPS shape is always:

- replace `return` with `k(...)`
- replace `y = f(...)` with `f_cps(..., lambda y: ...)`
- nest lambdas to represent "what to do next".

This is the **general scheme**: for each step, we introduce a continuation
describing the remaining steps.

## 3. Exceptions via CPS (success + error continuations)

We can extend CPS to carry **two continuations**:

- `ret` for success
- `err` for failure

A computation is now a function:

```python
(ret, err) -> None
```

It either calls `ret(value)` or `err(error)`.

In [4]:
# A success and failure representation
def ok(x):
    return lambda ret, err: ret(x)

def fail(msg):
    return lambda ret, err: err(msg)

# A bind combinator for these CPS values
def bind(op1, op2):
    """
    op1 : CPS value
    op2 : function from result -> CPS value

    bind(op1, op2) runs op1; if it succeeds with v, runs op2(v).
    Errors are propagated.
    """
    return lambda ret, err: op1(
        lambda v: op2(v)(ret, err),
        err
    )

# Example: a safe division and a sqrt that may fail
def safe_div(a, b):
    if b == 0:
        return fail("division by zero")
    return ok(a / b)

def sqrt_nonneg(x):
    if x < 0:
        return fail("negative input")
    return ok(x ** 0.5)


def example_pipeline(x, y):
    # direct-style idea:
    #   t = safe_div(x, y)
    #   u = sqrt_nonneg(t)
    #   return u
    #
    # CPS-style sequencing via bind:
    comp = bind(
        safe_div(x, y),
        lambda t: sqrt_nonneg(t)
    )
    return comp


def on_ok(v):
    print("OK:", v)

def on_err(e):
    print("ERROR:", e)

example_pipeline(10, 2)(on_ok, on_err)
example_pipeline(10, 0)(on_ok, on_err)

OK: 2.23606797749979
ERROR: division by zero


Here:

- `ok(x)` and `fail(msg)` construct CPS computations.
- `bind` is a **combinator** that wires two CPS computations together.
- The rules of error propagation live only inside `bind`.

This is a CPS-based model of exceptions: instead of raising an exception,
we call `err(...)`.

## 4. Backtracking via CPS (searching with multiple choices)

Backtracking algorithms explore alternatives and may need to **try another
option** when one branch fails. CPS with continuations makes this explicit.

We start with a very small example: search for an even number in a list,
with backtracking over choices.

In [5]:
def search_even_direct(xs):
    """Return the first even element in xs, or None."""
    for x in xs:
        if x % 2 == 0:
            return x
    return None


# CPS with success + failure + "alternative" continuation
def search_even_cps(xs, ret, err):
    """
    xs   : list of ints
    ret  : success continuation, called with found value
    err  : failure continuation, called if no even element
    """
    def loop(i):
        if i == len(xs):
            return err("no even element")
        x = xs[i]
        if x % 2 == 0:
            return ret(x)
        else:
            # try next element
            return loop(i + 1)
    return loop(0)


def ok(v):
    print("found even:", v)

def fail(msg):
    print("FAILED:", msg)

search_even_cps([1, 3, 5, 8, 9], ok, fail)
search_even_cps([1, 3, 5], ok, fail)

found even: 8
FAILED: no even element


In this small example, backtracking is simple: the only "choice" is which
index to try next. For more complex searches (trees, puzzles, n-Queens),
CPS lets us represent:

- "what to do if this branch succeeds" (`ret`)
- "what to do if this branch fails" (`err` or a continuation that tries
  another branch)

The n-Queens example and logic-programming experiments in the original
note are larger-scale applications of this same pattern.

## 5. Summary

- CPS makes "the rest of the computation" explicit as a function
  (the continuation).
- A CPS function does not return; it calls its continuation instead.
- We can **mechanically translate** direct-style code into CPS by:
  - adding an extra continuation parameter,
  - replacing `return e` with "evaluate `e` then call `k` on the result",
  - replacing `y = f(...)` with `f_cps(..., lambda y: ...)`.
- By using multiple continuations (success, error), we can model
  exceptions and backtracking in a purely functional way.
- Combinators like `bind` capture common wiring patterns so that user
  code remains readable.

This simplified CPS view is meant as a *low-level appendix* to the main
generator/fork continuation lecture. It explains how control flow can be
reduced to function calls and how higher-level control constructs can be
implemented without new syntax.